In [1]:
import requests
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from dotenv import load_dotenv

In [32]:
 # Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv('NOAA_CLIMATE_DATA')
token = api_key

In [22]:
def get_weather_data(token, location_id, start_date, end_date, data_type):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': token
    }
    params = {
        'datasetid': 'GSOY',  # Global Historical Climatology Network - GSOM is monthly, GSOY is yearly
        'locationid': f'FIPS:{location_id}',  # Location ID for country
        'startdate': start_date,
        'enddate': end_date,
        'datatypeid': data_type,
        'units': 'standard',  # Use 'standard' for Fahrenheit, inches, etc.
        'limit': 1000  # Adjust as needed
    }

    response = requests.get(url, headers=headers, params=params)
    
    return response



In [4]:
# list of dates
start_dates = [
    '1995-01-01', '2000-01-01', '2005-01-01', '2010-01-01',
    '2015-01-01', '2020-01-01', '2023-01-01'
]



end_dates = [  
    '1999-12-31', '2004-12-31', '2009-12-31', '2014-12-31',
    '2019-12-31', '2023-12-31'
]

# Date types to get
data_types = ["DP10", "DP1X", "DT32", "DX70", "DX90", "PRCP", "RHAV", "TAVG", 'TMAX', 'EMXT', 'EMNT', 'HTDD', 'EMXP', 'MNPN','MXPN', 'HXyz', 'HNyz']

In [5]:
# Load country codes to data frame
codes_path = 'Resources/ghcnd_country_codes.csv'
code_country_df = pd.read_csv(codes_path, sep=',', header=0)
location_ids = code_country_df['Code'].tolist()
print(code_country_df.tail())

    Code
213   WI
214   WQ
215   WZ
216   ZA
217   ZI


In [29]:
# Loop through the country codes
import time

weather_data_df = pd.DataFrame()
for country in code_country_df['Code']:
    # For each country
    print(country)
    
    # For each year set (start - end)
    for start_date, end_date in zip(start_dates, end_dates):
        weather_data = get_weather_data(token, country, start_date, end_date, data_types)
        if weather_data.status_code == 200:
            weather_response = weather_data.json()
            if 'results' in weather_response:
                weather_data = pd.DataFrame(weather_data.json()['results'])
                weather_data['country'] = country
                weather_data_df = pd.concat([weather_data_df, weather_data])

weather_data_df

AE
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AF
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AG
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AJ
<Response [200]>
<Response [200]>
<Response [200]>
<Response [503]>
<Response [200]>
<Response [200]>
AL
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AM
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AO
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AQ
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AR
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
AS
<Response [200]>
<Response [503]>
<Response [200]>
<

,date,datatype,station,attributes,value,country
0,1996-01-01T00:00:00,DX70,GHCND:AEM00041218,S,339.0,AE
1,1996-01-01T00:00:00,DX90,GHCND:AEM00041218,S,218.0,AE
2,1996-01-01T00:00:00,EMXT,GHCND:AEM00041218,"S,0706,",119.0,AE
3,1996-01-01T00:00:00,TMAX,GHCND:AEM00041218,S,96.7,AE
0,2004-01-01T00:00:00,DP10,GHCND:AEM00041194,S,4.0,AE
...,...,...,...,...,...,...
3,2022-01-01T00:00:00,TMAX,GHCND:WA007401540,S,84.2,WA
4,2022-01-01T00:00:00,DX70,GHCND:WA010101860,S,348.0,WA
5,2022-01-01T00:00:00,DX90,GHCND:WA010101860,S,126.0,WA
6,2022-01-01T00:00:00,EMXT,GHCND:WA010101860,"S,1004,",99.0,WA


In [31]:
weather_data_df.to_csv('Resources/weather_all_country_codes3.csv')